<a href="https://colab.research.google.com/github/TorbjornLarsson/SCDA/blob/main/Lab2_P4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab exercise 2: Regression, Least squares, QR

## Part 4: QR factorization and application to least squares problem


This part of the lab the QR factorization in Python, and its application for solving the least squares problem.


### QR factorization

Every matrix $A\in \mathbb R^{m\times n}$ with $m\geqslant n$ (overdetermined) can be factorized as
$$
A = QR
$$
where $Q$ is a $m\times m$ **orthogonal** matrix and $R$ is a $m\times n$ **upper triangular** matrix.

$$
\begin{array}{cccc}
\begin{pmatrix}
  \times & \times & \times  \\
  \times & \times & \times  \\
  \times & \times & \times  \\
  \times & \times & \times  \\
  \times & \times & \times
\end{pmatrix}&=&
\begin{pmatrix}
  \times & \times & \times & \times & \times \\
  \times & \times & \times & \times & \times \\
  \times & \times & \times & \times & \times \\
  \times & \times & \times & \times & \times \\
  \times & \times & \times & \times & \times
\end{pmatrix}&
\begin{pmatrix}
\times & \times  & \times\\
 0 & \times  & \times\\
 0 &  0  & \times \\ \hline
 0 & 0 & 0\\
  0 & 0 & 0
\end{pmatrix}
\\ \\
A&& Q&R
\end{array}
$$

As we see $m-n$ last rows of $R$ are zero which role out the last $m-n$ columns of $Q$, enable us to write the thin (or reduced) QR factorization
$$
A = Q_1 R_1
$$
where $Q = [Q_1\;Q_2]$ and $R = \begin{bmatrix} R_1\\ 0 \end{bmatrix}$. Here $Q_1$ consists of the first $n$ columns of $Q$.

In numpy, the command `Q, R = np.linalg.qr(A, mode = 'complete')` computes the complete QR factorization of the matrix $A$. As an example, in the cell below compute and print the Q and R factors of the following matrix:
$$
A = \begin{bmatrix}
1&-1&1\\
1&1&1\\
1&2&4\\
1&3&9\\
1&4&16
\end{bmatrix}
$$
This is the coefficient matrix of a quadratic polynomial fitting on points $-1,1,2,3,4$.

In [1]:
import numpy as np

A = np.array([[1, -1, 1],
              [1, 1, 1],
              [1, 2, 4],
              [1, 3, 9],
              [1, 4, 16]])

Q, R = np.linalg.qr(A, mode = 'complete')

print("Q factor:\n", Q)
print("\nR factor:\n", R)

Q factor:
 [[-0.4472136   0.72782534  0.49072924  0.14646213  0.08946465]
 [-0.4472136   0.2079501  -0.47288454 -0.47660981 -0.55315464]
 [-0.4472136  -0.05198752 -0.45950102 -0.03479192  0.76481739]
 [-0.4472136  -0.31192515 -0.11599055  0.76710259 -0.31749411]
 [-0.4472136  -0.57186277  0.55764687 -0.402163    0.01636672]]

R factor:
 [[ -2.23606798  -4.02492236 -13.86362146]
 [  0.          -3.84707681 -11.22930529]
 [  0.           0.           6.05827556]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]]


Changing the `mode` to `mode = 'reduced'` gives the reduced QR factorization. Compute and print the reduced QR factorization of the above matrix in the cell below:

In [2]:
Q_reduced, R_reduced = np.linalg.qr(A, mode = 'reduced')

print("Reduced Q factor:\n", Q_reduced)
print("\nReduced R factor:\n", R_reduced)

Reduced Q factor:
 [[-0.4472136   0.72782534  0.49072924]
 [-0.4472136   0.2079501  -0.47288454]
 [-0.4472136  -0.05198752 -0.45950102]
 [-0.4472136  -0.31192515 -0.11599055]
 [-0.4472136  -0.57186277  0.55764687]]

Reduced R factor:
 [[ -2.23606798  -4.02492236 -13.86362146]
 [  0.          -3.84707681 -11.22930529]
 [  0.           0.           6.05827556]]


### Computing $Q$ and $R$ using Householder reflections

Given a non-zero vector $u$ in $\mathbb R^n$, a matrix of the form
\begin{equation*}
  H = I -\frac{2}{u^Tu}uu^T
\end{equation*}
is called a **Householder matrix** or a **Householder reflection**.

Matrix $H$ is orthogonal and can be used to iteratively introduce zeros under the diagonals of matrix $A$ ultimately leading to the QR factorization of $A$. Let's illustrate this process with an example:

Consder the matrix $A$ above and let $x$ be its first column, i.e.
$$
x = \begin{bmatrix}
1\\ 1\\ 1\\ 1\\ 1
\end{bmatrix}
$$
Now set $u = x + \mathrm{sign}(x_1)\|x\|_2 e_1$ where $e_1 = [1,0,0,0,0]^T$, and then compute the Householder matrix $H_1 = I-(2/u^Tu)uu^T$.
The code is given below:

In [3]:
import numpy as np

def HouseVec(x):
  e = np.zeros(len(x)); e[0] = 1
  s = -1 if x[0] < 0  else 1      # our sign function (it is different from np.sign(a) as np.sign(0) = 0)
  u = x + s*np.linalg.norm(x,2)*e
  return u

A = np.array([[1,-1,1],[1,1,1],[1,2,4],[1,3,9],[1,4,16]])
x = A[:,0] # first column
n = len(x)
u = HouseVec(x)
H1 = np.eye(n) - 2/np.dot(u,u) * np.outer(u,u)

Multiply $H_1$ by $A$ to get $A_1$: (for matrix-matrix or matrix-vector multiplication you can use `@` operator or `np.matmul()`, or `np.dot()` commands. Avoid using `*` operator.)

In [4]:
A1 = H1 @ A
print("A1 factor:\n", A1)

A1 factor:
 [[-2.23606798e+00 -4.02492236e+00 -1.38636215e+01]
 [ 5.55111512e-17  6.52475842e-02 -3.59311163e+00]
 [ 5.55111512e-17  1.06524758e+00 -5.93111629e-01]
 [ 2.77555756e-17  2.06524758e+00  4.40688837e+00]
 [ 0.00000000e+00  3.06524758e+00  1.14068884e+01]]


What does $A_1$ look like? For the second step, let's assume that $x$ is a $4\times 1$ vector located below the diagonal in the second column of $A_1$, including the diagonal elements. Now, proceed with the same procedure to compute the Householder matrix $\widetilde{H}_2$. Finally, construct the matrix $H_2$ as follows:
$$
H_2 = \begin{bmatrix}
1& 0 \\ 0 & \widetilde H_2
\end{bmatrix}.
$$
The code is given below:

In [5]:
x = A1[1:,1]  # second column diagonal and under diagonal
u = HouseVec(x)
Ht = np.eye(n-1) - 2/np.dot(u,u) * np.outer(u,u)
H2 = np.block([[np.ones(1), np.zeros([1,n-1])],[np.zeros([n-1,1]), Ht]])

Note that `np.block` is used to form a block matrix. It works similar to `np.array` by replacing matrices instead of individual numbers.

Now compute and print the new matrix $A_2 = H_2A_1$ in the cell below. What do you observe?

In [6]:
A2 = H2 @ A1
print("A2 factor:\n", A2)

A2 factor:
 [[-2.23606798e+00 -4.02492236e+00 -1.38636215e+01]
 [-3.12125881e-17 -3.84707681e+00 -1.12293053e+01]
 [ 3.18980138e-17  3.96866716e-18 -2.67229424e+00]
 [-1.80243687e-17  2.32646502e-16  3.75875369e-01]
 [-6.79467512e-17  1.46958084e-16  5.42404498e+00]]


Continue with this process to the third step and calculate matrix $A_3$ as the product of $H_3$ and $A_2$. It's important to note that this resulting matrix should be upper triangular, representing the $R$ factor in the QR factorization. The $Q$ factor is obtained as the product of $H_1$, $H_2$, and $H_3$. Finalize your code and compute both factors accordingly:

In [7]:
x = A2[2:,2] # third column diagonal and under diagonal
n_prime = len(x)
u = HouseVec(x)
Ht = np.eye(n_prime) - 2/np.dot(u,u) * np.outer(u,u)

H3 = np.block([
    [np.eye(2), np.zeros([2, n_prime])],
    [np.zeros([n_prime, 2]), Ht]
])

A3 = H3 @ A2
print("A3 (R factor):\n", A3)

# Calculate the Q factor
Q_householder = H1 @ H2 @ H3
print("\nQ factor (from Householder):\n", Q_householder)

A3 (R factor):
 [[-2.23606798e+00 -4.02492236e+00 -1.38636215e+01]
 [-3.12125881e-17 -3.84707681e+00 -1.12293053e+01]
 [-7.60219678e-17  1.44256875e-16  6.05827556e+00]
 [-1.33781126e-17  2.26606704e-16  9.49346447e-17]
 [-8.99256276e-19  5.98011667e-17  1.20520616e-15]]

Q factor (from Householder):
 [[-0.4472136   0.72782534  0.49072924  0.14646213  0.08946465]
 [-0.4472136   0.2079501  -0.47288454 -0.47660981 -0.55315464]
 [-0.4472136  -0.05198752 -0.45950102 -0.03479192  0.76481739]
 [-0.4472136  -0.31192515 -0.11599055  0.76710259 -0.31749411]
 [-0.4472136  -0.57186277  0.55764687 -0.402163    0.01636672]]


Compare your computed factors $Q$ and $R$ with those of the `np.linalg.qr` command:

In [8]:
print("Comparing Q factors:")
print("Are Q_householder and Q (from np.linalg.qr) close?", np.allclose(Q_householder, Q))

print("\nComparing R factors:")
# For R, we compare A3 with the upper 3x3 part of R from np.linalg.qr,
# as A3 is the effective R factor (upper triangular part).
# We need to extract the corresponding part from the R obtained by np.linalg.qr
R_from_householder = A3[:3, :3] # Extract the upper triangular part for comparison
R_from_linalg_qr = R[:3, :3] # Extract the upper triangular part from np.linalg.qr

print("Are R (from Householder) and R (from np.linalg.qr) close?", np.allclose(R_from_householder, R_from_linalg_qr))

# Also compare the full R factors, noting that A3 might have near-zero values below the diagonal
# while R from np.linalg.qr has exact zeros.
# We can check if the upper triangular part is close and if the lower part of A3 is near zero.
print("\nComparing full A3 (R factor from Householder) with R from np.linalg.qr (up to numerical precision):")
print(np.allclose(A3, R, atol=1e-9))

Comparing Q factors:
Are Q_householder and Q (from np.linalg.qr) close? True

Comparing R factors:
Are R (from Householder) and R (from np.linalg.qr) close? True

Comparing full A3 (R factor from Householder) with R from np.linalg.qr (up to numerical precision):
True


### Application to solve the least squares problem

For a full-rank matrix $A$, the solution of the least square problem
$$
\mbox{find}~~ x\in\mathbb R^n ~~\mbox{such that }~~ \|Ax-b\|_2 ~~\mbox{is mimimized}
$$
is obtained by solving the triangular system
$$R_1x = Q_1^Tb$$
using a backward substitution, and the residual is
$$residual = \|Ax-b\|_2=\|Q_2^Tb\|_2.$$
Assume that $A$ and $b$ are given as
$$
A = \begin{bmatrix}
1&-1&1\\
1&1&1\\
1&2&4\\
1&3&9\\
1&4&16
\end{bmatrix}
,\qquad
b = \begin{bmatrix}
1\\ 2\\ 1\\ 2\\ 3
\end{bmatrix}.
$$
Write a code to compute the least square solution of $Ax\approx b$ using the QR factorization and compute the residual:

In [9]:
import numpy as np

# Define matrix A and vector b
A = np.array([[1,-1,1],
              [1,1,1],
              [1,2,4],
              [1,3,9],
              [1,4,16]])

b = np.array([[1],
              [2],
              [1],
              [2],
              [3]])

# Perform complete QR factorization
Q, R = np.linalg.qr(A, mode='complete')

# Get dimensions
m, n = A.shape

# Extract R1 (upper n x n part of R)
R1 = R[:n, :n]

# Extract Q1 (first n columns of Q)
Q1 = Q[:, :n]

# Extract Q2 (remaining m-n columns of Q, if any)
# If m == n, Q2 will be empty
Q2 = Q[:, n:]

# Solve R1x = Q1^T b for x
x = np.linalg.solve(R1, Q1.T @ b)

# Compute the residual ||Ax-b||_2 = ||Q2^T b||_2
# Check if Q2 is empty (case where m = n and exact solution exists)
if Q2.shape[1] > 0:
    residual = np.linalg.norm(Q2.T @ b)
else:
    # If m == n, there is no Q2 component for the residual calculation
    # and the residual should ideally be zero for a non-singular A.
    residual = np.linalg.norm(A @ x - b)

print("Least squares solution x:\n", x)
print("\nResidual (||Ax-b||_2):", residual)

Least squares solution x:
 [[1.13402062]
 [0.07069219]
 [0.08689249]]

Residual (||Ax-b||_2): 0.982917421174488
